In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle.json': b'{"username":"ahmedelkhamisi","key":"f9f4c8dfa48d84d6567952071d66901a"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d vikrishnan/boston-house-prices

boston-house-prices.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from zipfile import ZipFile
file_name="boston-house-prices.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import numpy as np,matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

%matplotlib inline

# Read Dataset

In [8]:
from sklearn.datasets import load_boston
boston=load_boston()
boston.feature_names

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [9]:
header=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 
        'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

df = pd.read_csv("housing.csv", names=header, delim_whitespace=True)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [10]:
df.shape

(506, 14)

In [11]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [12]:
#Preprocessing (scaling)
scaler = preprocessing.StandardScaler()
scaler.fit(df.drop('MEDV',axis=1))
scaled_features = scaler.transform(df.drop('MEDV',axis=1))
x = PolynomialFeatures(degree=1)
data= x.fit_transform(scaled_features)

In [13]:
#adding bias column
df_final = pd.DataFrame(data,columns= ["bias","CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"])

In [14]:
df_final.head()

,bias,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,1.0,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,1.0,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,1.0,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,1.0,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


In [15]:
Y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(df_final, Y, test_size=0.30 , random_state=1)
X_train, X_cv, y_train, y_cv = train_test_split(df_final, Y, test_size=0.25 , random_state=1)


In [16]:
print(X_train.shape)
print(X_test.shape)

(379, 14)
(152, 14)


In [17]:
print(y_train.shape)
print(y_test.shape)

(379,)
(152,)


In [18]:
#Fitting model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

In [19]:
#Prediction
y_pred = lin_model.predict(X_test)
y_pred

array([32.37816533, 27.95684437, 18.07213891, 21.63166556, 18.93029508,
       19.96277202, 32.2834674 , 18.06715668, 24.72989076, 26.85359369,
       27.23326816, 28.57021239, 21.18778302, 26.94393815, 23.37892579,
       20.89176865, 17.11746934, 37.73997945, 30.51980066,  8.44489436,
       20.86557977, 16.21989418, 25.13605925, 24.77658813, 31.40497629,
       11.02741407, 13.82097563, 16.80208261, 35.94637198, 14.7155729 ,
       21.23939821, 14.15079469, 42.72492585, 17.83887162, 21.84610225,
       20.40178099, 17.50287927, 27.00093206,  9.80760408, 20.00288662,
       24.27066782, 21.06719021, 29.47089776, 16.48482565, 19.38852695,
       14.54778282, 39.39838319, 18.09810655, 26.22164983, 20.60676525,
       25.09994066, 24.48366723, 25.02297948, 26.84986898,  5.01517985,
       24.12809513, 10.72843392, 26.83178157, 16.8023533 , 35.48142073,
       19.50937911, 27.43260347, 16.58016763, 19.151488  , 10.9990262 ,
       32.05016535, 36.32672849, 21.8596379 , 24.8158357 , 25.32

In [20]:
mse = mean_squared_error(y_test,y_pred)
mse

19.521867950469975

In [21]:
accuracy=lin_model.score(X_test,y_test)
accuracy

0.7870058677128697

In [22]:
#predicting on validation
val_p=lin_model.predict(X_cv)
val_p

array([32.37816533, 27.95684437, 18.07213891, 21.63166556, 18.93029508,
       19.96277202, 32.2834674 , 18.06715668, 24.72989076, 26.85359369,
       27.23326816, 28.57021239, 21.18778302, 26.94393815, 23.37892579,
       20.89176865, 17.11746934, 37.73997945, 30.51980066,  8.44489436,
       20.86557977, 16.21989418, 25.13605925, 24.77658813, 31.40497629,
       11.02741407, 13.82097563, 16.80208261, 35.94637198, 14.7155729 ,
       21.23939821, 14.15079469, 42.72492585, 17.83887162, 21.84610225,
       20.40178099, 17.50287927, 27.00093206,  9.80760408, 20.00288662,
       24.27066782, 21.06719021, 29.47089776, 16.48482565, 19.38852695,
       14.54778282, 39.39838319, 18.09810655, 26.22164983, 20.60676525,
       25.09994066, 24.48366723, 25.02297948, 26.84986898,  5.01517985,
       24.12809513, 10.72843392, 26.83178157, 16.8023533 , 35.48142073,
       19.50937911, 27.43260347, 16.58016763, 19.151488  , 10.9990262 ,
       32.05016535, 36.32672849, 21.8596379 , 24.8158357 , 25.32

In [23]:
v_acc=lin_model.score(X_cv,y_cv)
v_acc

0.7789410172622858

In [24]:
##Testing hypothesis for different degrees using Polynomial features

In [25]:
MSE=[] #To store values of MSE
Z =[1,2,3,4,5,6] #degrees
for i in Z:
    
    poly_reg = PolynomialFeatures(degree=i)
    X_p = poly_reg.fit_transform(X_train)
    pol_reg = linear_model.LinearRegression()
    model=(pol_reg.fit(X_p,y_train))
    
    x_pv=poly_reg.fit_transform(X_cv)
    predict_new=model.predict(x_pv)
    MSE.append(mean_squared_error(y_cv, predict_new))  

In [26]:
#Getting least error
Z[np.argmin(MSE)]

2

In [44]:
#Using Ridge (Regularization)
lamda=[0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
errors=[]
for j in lamda:
  
    ridge=Ridge(alpha=j)
    poly_reg = PolynomialFeatures(degree=2)
    X_ridge = poly_reg.fit_transform(X_train)
    model_r=ridge.fit(X_ridge,y_train)
    x_cv_ridge=poly_reg.fit_transform(X_cv)
    Y_predict_r=model_r.predict(x_cv_ridge)
    errors.append(mean_squared_error(y_cv, Y_predict_r))


In [45]:
errors

[13.172495078740159,
 10.316337543155774,
 10.252418570373255,
 10.147092223522025,
 10.017657660787897,
 9.887340467698364,
 9.756074685510459,
 9.607424349904495,
 9.444944536371146,
 9.288122030101727,
 9.122340378646326,
 8.905435469855465]

In [46]:
lamda[np.argmin(errors)]

10.24

In [47]:
Y_predict_r

array([31.66904344, 25.64686556, 19.21548623, 19.37494392, 25.4018869 ,
       20.32955532, 29.40846676, 16.40438922, 21.39599108, 26.26329159,
       28.5111349 , 29.21077833, 19.05779422, 16.99537474, 20.24248522,
       25.60264219, 11.44423878, 44.56570884, 21.28432747, 13.61647919,
       17.82776798, 15.92629077, 23.43982921, 24.9336329 , 24.0235279 ,
       10.54869322, 14.03397173, 20.50874907, 38.82554083, 15.94382707,
       25.70344499, 16.73249381, 51.80046378, 15.11231599, 21.57994272,
       22.89906732, 15.35920538, 30.9670905 , 16.06774469, 18.96447749,
       25.20420137, 23.12181971, 27.81225166, 14.42869914, 17.48473394,
       10.32838965, 49.23953118, 17.18746224, 23.73885543, 13.77042606,
       24.13137256, 21.77580237, 24.90720338, 25.43040954,  9.74509374,
       22.62482377, 11.06707494, 25.23968013, 16.89249952, 41.5502119 ,
       18.00893792, 27.27017923, 16.29434627, 18.63754714, 31.08904557,
       35.14189803, 43.0493159 , 22.93240148, 21.02021624, 21.50

In [48]:
mean_squared_error(y_cv, Y_predict_r)

8.905435469855465